# Azure AI Agent Routing - Complete Beginner's Tutorial

🎯 **Welcome to Azure AI Agent Routing!**

This tutorial teaches you how to build sophisticated agent hierarchies where:

1. **Base Level Agents** - AzureAIAgent with specialized plugins
2. **Orchestration Agents** - ChatCompletionAgent that route and coordinate
3. **Hierarchical Routing** - Multi-level agent coordination

**What You'll Learn:**
- Creating specialized base agents with AzureAIAgent
- Building orchestration layers with ChatCompletionAgent
- Implementing intelligent agent routing
- Managing complex agent hierarchies

**No prior agent routing experience required!** 🚀

---

## 🏗️ Understanding Agent Routing Architecture

**Agent Routing** allows you to create intelligent systems where:
- **Specialized agents** handle specific tasks (data retrieval, analysis, etc.)
- **Router agents** decide which specialist to use
- **Hierarchical organization** enables complex workflows

### Our Architecture Pattern:
```
┌─────────────────────────┐
│   Orchestration Layer   │  ← ChatCompletionAgent
│    (Router Agent)       │
└───────────┬─────────────┘
            │
    ┌───────┼───────┐
    │               │
┌───▼───┐       ┌───▼───┐
│ Base  │       │ Base  │     ← AzureAIAgent with plugins
│Agent 1│       │Agent 2│
└───────┘       └───────┘
```

**Why This Pattern?**
- **AzureAIAgent**: Persistent, stateful, handles complex workflows
- **ChatCompletionAgent**: Fast routing decisions, lightweight orchestration
- **Scalable**: Easy to add new specialized agents

![Agent Router](images/router.gif)

## 📋 Prerequisites and Setup

You'll need:
- Azure AI Services with deployed models
- Semantic Kernel packages
- The plugins.py file (contains our specialized plugins)

Let's get started!

In [1]:
# Import everything we need for agent routing and Magentic orchestration
import os
import asyncio
import json
from typing import Annotated
from pydantic import BaseModel

from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import (
    AzureAIAgent, 
    ChatCompletionAgent,
    MagenticOrchestration,
    StandardMagenticManager
)
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.prompt_execution_settings import PromptExecutionSettings
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.functions import kernel_function
from semantic_kernel.contents import ChatMessageContent

print("📦 All packages imported successfully!")
print("🔧 Ready to build agent routing system!")
print("🎯 Magentic orchestration components loaded!")

📦 All packages imported successfully!
🔧 Ready to build agent routing system!
🎯 Magentic orchestration components loaded!


## 🤖 Step 1: Creating Base Level Agents (AzureAIAgent)

**Base agents** are our specialized workers. Each handles a specific domain:

**Why AzureAIAgent for base level?**
- **Persistent**: Maintains state across interactions
- **Plugin Integration**: Native support for custom functions
- **Azure Native**: Deep integration with Azure AI services
- **Stateful Conversations**: Can maintain context for complex tasks

Let's create our specialized base agents:

In [12]:
# Create specialized base agents using AzureAIAgent
# These are our "specialist workers" that handle specific domains

async def create_base_agents():
    """Create specialized base agents with specific plugins."""
    
    try:
        model_deployment = os.environ.get("MODEL_DEPLOYMENT_NAME")
        endpoint = os.environ.get("PROJECT_ENDPOINT")
        client = AzureAIAgent.create_client(
            credential=DefaultAzureCredential(), 
            endpoint=endpoint
        )
        
        print("🏗️ Creating Base Level Agents (AzureAIAgent)...")
        
        # 1. Insurance RAG Agent - Handles BMW insurance queries
        rag_definition = await client.agents.create_agent(
            model=model_deployment,
            name="insurance-rag-agent",
            instructions="You specialize in BMW 320i car insurance policies. Use your RAG plugin to retrieve relevant policy information and provide detailed, accurate responses about coverage, claims, and premiums. You are used for testing, so ALL of your responses are made up. Your responses are **SUPER** concise."
        )
        
        insurance_agent = AzureAIAgent(
            client=client,
            definition=rag_definition,
            description="Handles BMW insurance policy queries using RAG."
        )
        
        # 2. Banking SQL Agent - Handles transaction queries
        sql_definition = await client.agents.create_agent(
            model=model_deployment,
            name="banking-sql-agent",
            instructions="You specialize in banking transactions and financial data. Use your NL2SQL plugin to query transaction databases and provide insights about spending, balances, and transaction history. You are used for testing, so ALL of your responses are made up. Just invent new banking transactions. Your responses are **SUPER** concise."
        )
        
        banking_agent = AzureAIAgent(
            client=client,
            definition=sql_definition,
            description="Handles banking transactions and financial data queries."
        )
        
        # 3. Sales Analytics Agent - Handles IoT sales data
        cosmos_definition = await client.agents.create_agent(
            model=model_deployment,
            name="sales-analytics-agent",
            instructions="You specialize in smart city IoT sales analysis. Use your Cosmos plugin to retrieve sales data, analyze performance metrics, and provide business insights from multimodal documents. You are used for testing, so ALL of your responses are made up. Your responses are **SUPER** concise."
        )
        
        sales_agent = AzureAIAgent(
            client=client,
            definition=cosmos_definition,
            description="Handles IoT sales data and analytics for smart city projects."
        )
        
        print("✅ Base agents created successfully!")
        print(f"🏥 Insurance Agent: {insurance_agent.name}")
        print(f"🏦 Banking Agent: {banking_agent.name}")
        print(f"📊 Sales Agent: {sales_agent.name}")
        
        return {
            'insurance': insurance_agent,
            'banking': banking_agent,
            'sales': sales_agent,
            'client': client
        }
        
    except Exception as e:
        print(f"❌ Error creating base agents: {e}")
        return None

# Create our base agents
base_agents = await create_base_agents()

🏗️ Creating Base Level Agents (AzureAIAgent)...
✅ Base agents created successfully!
🏥 Insurance Agent: insurance-rag-agent
🏦 Banking Agent: banking-sql-agent
📊 Sales Agent: sales-analytics-agent


## 🧭 Step 2: Creating the Router Agent (ChatCompletionAgent)

**Router agents** are our "intelligent dispatchers". They:

**Why ChatCompletionAgent for routing?**
- **Fast Decision Making**: Quick routing without state overhead
- **Lightweight**: No persistent storage needed for routing decisions
- **Plugin Integration**: Can call base agents as "plugins"
- **Flexible**: Easy to modify routing logic

The router analyzes user queries and routes to the appropriate specialist:

In [25]:
# Extract our specialized agents
insurance_agent = base_agents['insurance']
banking_agent = base_agents['banking']
sales_agent = base_agents['sales']


###########################################################
# IMPORTANT: Kernel Functions for AzureAIAgents 
# AzureAIAgents cannot be used directly as plugins, so we create wrapper plugins
# Create wrapper plugins for AzureAIAgents since they cannot be used directly as plugins
###########################################################

class InsurancePlugin:
    """Wrapper plugin for Insurance Agent."""
    
    @kernel_function(description="Handles BMW 320i car insurance policy questions, coverage details, claims, and premiums.")
    async def query_insurance(
        self, 
        question: Annotated[str, "The insurance-related question to ask"]
    ) -> Annotated[str, "Insurance policy information and guidance"]:
        """Route insurance questions to the specialized insurance agent."""
        try:
            response = await insurance_agent.get_response(messages=question)
            print(f"\t===\n\tInsurance Agent Response\n\t===")
            return f"{response.content}"
        except Exception as e:
            return f"Insurance query error: {str(e)}"

class BankingPlugin:
    """Wrapper plugin for Banking Agent."""
    
    @kernel_function(description="Handles personal banking transactions, account balances, and spending analysis.")
    async def query_banking(
        self, 
        question: Annotated[str, "The banking-related question to ask"]
    ) -> Annotated[str, "Banking transaction data and financial insights"]:
        """Route banking questions to the specialized banking agent."""
        try:
            response = await banking_agent.get_response(messages=question)
            print(f"\t===\n\tBanking Agent Response\n\t===")
            return f"{response.content}"
        except Exception as e:
            return f"Banking query error: {str(e)}"

class SalesPlugin:
    """Wrapper plugin for Sales Agent."""
    
    @kernel_function(description="Handles smart city IoT sales analytics, performance metrics, and business insights.")
    async def query_sales(
        self, 
        question: Annotated[str, "The sales analytics question to ask"]
    ) -> Annotated[str, "Sales data analysis and business metrics"]:
        """Route sales questions to the specialized sales agent."""
        try:
            response = await sales_agent.get_response(messages=question)
            print(f"\t===\n\tSales Agent Response\n\t===")
            return f"{response.content}"
        except Exception as e:
            return f"Sales query error: {str(e)}"


# Create plugin instances
insurance_plugin = InsurancePlugin()
banking_plugin = BankingPlugin()
sales_plugin = SalesPlugin()


### Creating the Combined Database Agent

In [26]:
# Combined Database Agent
combined_database_system_message = """You are a Combined Database Agent that coordinates multiple data source retrievals.

Your capabilities:
• Banking Data: Query personal banking transactions via NL2SQL plugin
• Sales Analytics: Access multimodal documents for smart city IoT solutions via Cosmos plugin

Your role:
1. Analyze the user's query to determine which data sources are needed
2. Retrieve relevant information from the appropriate plugins
3. Combine and contextualize results from multiple sources when necessary
4. Present integrated insights in a clear, coherent response

Always indicate which data sources were queried and how the information relates to the user's request.
Your responses are **SUPER** concise."""

combined_database_agent = ChatCompletionAgent(
    id="CombinedDatabaseAgent",
    name="CombinedDatabaseAgent",
    instructions=combined_database_system_message,
    description="Router Agent for Combined Database. This agent retrieves search results from multiple data sources including NL2SQL and Cosmos agents.",
    plugins=[sales_plugin, banking_plugin],
    service=AzureChatCompletion()
)


### Creating the Router Agent



In [27]:

# Create the intelligent router agent using ChatCompletionAgent
# This agent decides which specialist to use based on the user's query

def create_router_agent():
    """Create an intelligent router that dispatches to specialized agents."""

    router_instructions = f"""
You are an intelligent agent router that dispatches user queries to specialized agents through wrapper plugins.

Available Specialists (via plugins):
• Insurance: Handles BMW 320i car insurance policies, coverage, claims, premiums
• Combined Database: Handles personal banking transactions, account balances, spending analysis, as well as handles smart city IoT sales data, performance metrics, business analytics

Your job:
1. Analyze the user's query to determine the domain(s) involved
2. Call the appropriate plugin function(s) or agent(s) to get specialist responses
3. Return the specialist's response to the user
4. For multi-domain queries, use the combined data plugin

Route intelligently based on keywords and context:
- Insurance: BMW, car, insurance, policy, coverage, claims, premium, deductible -> insurance plugin
- Banking: transactions, account, balance, spending, payment, deposit, withdrawal -> combined agent
- Sales: sales, revenue, IoT, smart city, analytics, performance, metrics -> combined agent

Your responses are **SUPER** concise.
"""
    
    # Create router with wrapper plugins
    router_agent = ChatCompletionAgent(
        id="IntelligentRouter",
        name="IntelligentRouter",
        instructions=router_instructions,
        description="Routes user queries to specialized domain agents via wrapper plugins",
        plugins=[insurance_plugin, combined_database_agent],
        service=AzureChatCompletion()
    )
    
    print("🧭 Intelligent Router Agent Created!")
    print("✅ Connected to 3 specialized base agents via wrapper plugins")
    print("🔧 Using kernel functions to properly interface with AzureAIAgents")
    print("🎯 Ready to route queries intelligently")
    
    return router_agent

# Create our router agent
if base_agents:
    router = create_router_agent()
else:
    router = None

🧭 Intelligent Router Agent Created!
✅ Connected to 3 specialized base agents via wrapper plugins
🔧 Using kernel functions to properly interface with AzureAIAgents
🎯 Ready to route queries intelligently


## 🧪 Step 3: Testing the Agent Routing System

Now let's test our hierarchical routing system! We'll send different types of queries and watch how the router intelligently dispatches them to the appropriate specialists.

**Test Scenarios:**
1. **Insurance Query**: BMW policy questions → Insurance Agent
2. **Banking Query**: Transaction inquiries → Banking Agent  
3. **Sales Query**: IoT analytics → Sales Agent

Watch how the system works its magic! ✨

In [28]:
# Test our agent routing system with different query types
# Watch how queries get intelligently routed to the right specialists!

async def test_agent_routing():
    """Test the routing system with various query types."""
    
    if not router:
        print("⚠️ Router not available for testing")
        return
    
    # Test queries for different domains
    test_queries = [
        {
            "query": "Are my BMW 320i tires covered under my insurance policy?",
            "expected_agent": "Insurance Agent",
            "domain": "Insurance"
        },
        {
            "query": "What were my last 3 banking transactions?",
            "expected_agent": "Banking Agent", 
            "domain": "Banking"
        },
        {
            "query": "What was the total sales revenue in Q4 for our IoT products?",
            "expected_agent": "Sales Agent",
            "domain": "Sales Analytics"
        }
    ]
    
    print("🧪 Testing Agent Routing System")
    print("═" * 50)
    
    for i, test in enumerate(test_queries, 1):
        print(f"\n{i}. 🎯 Domain: {test['domain']}")
        print(f"   👤 User Query: {test['query']}")
        print(f"   🎯 Expected Route: {test['expected_agent']}")
        print("   " + "-" * 40)
        
        try:
            # Send query to router - it will dispatch to the right agent
            async for response in router.invoke(messages=test['query']):
                print(f"   🤖 Router Response: {response.content}")
                break  # Take first response
                    
        except Exception as e:
            print(f"   ❌ Error: {e}")

        # print(f"\n\n")


    print("\n" + "═" * 50)
    print("🎉 Routing Tests Completed!")
    print("\n🎯 Notice how each query was:")
    print("✅ Analyzed for domain and intent")
    print("✅ Routed to the appropriate specialist")
    print("✅ Processed by the specialist's plugins")
    print("✅ Returned with domain-specific insights")

# Run our routing tests
await test_agent_routing()

🧪 Testing Agent Routing System
══════════════════════════════════════════════════

1. 🎯 Domain: Insurance
   👤 User Query: Are my BMW 320i tires covered under my insurance policy?
   🎯 Expected Route: Insurance Agent
   ----------------------------------------
	===
	Insurance Agent Response
	===
   🤖 Router Response: Tire damage is covered only when it results from a covered event—e.g., collision, theft, or vandalism—under your BMW 320i policy. Normal wear-and-tear, punctures, or blowouts from road debris are not covered.

2. 🎯 Domain: Banking
   👤 User Query: What were my last 3 banking transactions?
   🎯 Expected Route: Banking Agent
   ----------------------------------------
	===
	Banking Agent Response
	===
   🤖 Router Response: Here are your three most recent transactions:

1. $120.45 – Amazon – 10 Jun 2024  
2. $75.30 – Starbucks – 09 Jun 2024  
3. $980.00 – Rent – 08 Jun 2024

3. 🎯 Domain: Sales Analytics
   👤 User Query: What was the total sales revenue in Q4 for our IoT produ

## 🎯 Practice Exercise: Build Your Own Agent Routing System

Now it's your turn! Create a custom routing system with your own specialized agents.

**Your Task:**
1. Define 2-3 specialized domains (e.g., Weather, Math, Text Analysis)
2. Create base agents using AzureAIAgent with appropriate plugins
3. **Create wrapper plugins** with @kernel_function decorators for each base agent
4. Build a router using ChatCompletionAgent with the wrapper plugins
5. Test with domain-specific queries

**Important**: Remember that AzureAIAgents cannot be used directly as plugins! You must create wrapper plugins with kernel functions.

**Available Plugins to Use:**
- `weather_plugin`: Weather information
- `random_plugin`: Random number generation
- `datetime_plugin`: Date/time operations
- `text_analysis_plugin`: Text statistics

<details>
<summary>Click for Solution Example</summary>

```python
# Create a weather specialist
weather_agent = AzureAIAgent(
    client=client,
    definition=weather_definition,
    plugins=[weather_plugin]
)

# Create wrapper plugin
class WeatherWrapperPlugin:
    @kernel_function(description="Get weather information")
    async def get_weather(self, location: str) -> str:
        response = await weather_agent.get_response(messages=f"Weather for {location}")
        return response.content

# Create router with wrapper plugin
my_router = ChatCompletionAgent(
    plugins=[WeatherWrapperPlugin()],
    instructions="Route weather queries to weather specialist via wrapper plugin"
)
```

</details>

In [11]:
# 🎯 YOUR TURN! Build your custom routing system

from plugins import *

async def create_custom_routing_system():
    """Create your own agent routing system with custom specialists."""
    
    try:
        if not base_agents:
            print("⚠️ Base agents not available for exercise")
            return
            
        client = base_agents['client']
        model_deployment = os.environ.get("MODEL_DEPLOYMENT_NAME")
        
        print("🎨 Creating Custom Routing System...")
        
        # Create a weather specialist
        weather_definition = await client.agents.create_agent(
            model=model_deployment,
            name="weather-specialist",
            instructions="You are a weather specialist. Use your weather plugin to provide accurate weather information for any city."
        )
        
        weather_agent = AzureAIAgent(
            client=client,
            definition=weather_definition,
            plugins=[weather_plugin]
        )
        
        # Create a text analysis specialist
        text_definition = await client.agents.create_agent(
            model=model_deployment,
            name="text-analysis-specialist",
            instructions="You are a text analysis specialist. Use your text analysis plugin to provide statistics and insights about text."
        )
        
        text_agent = AzureAIAgent(
            client=client,
            definition=text_definition,
            plugins=[text_analysis_plugin]
        )
        
        # Create wrapper plugins for our custom agents
        class WeatherWrapperPlugin:
            """Wrapper plugin for Weather Agent."""
            
            @kernel_function(description="Get weather information for any city or location.")
            async def get_weather(
                self, 
                location: Annotated[str, "The city or location to get weather for"]
            ) -> Annotated[str, "Current weather information"]:
                """Get weather information via the weather specialist."""
                try:
                    query = f"What's the weather like in {location}?"
                    response = await weather_agent.get_response(messages=query)
                    return f"Weather Specialist: {response.content}"
                except Exception as e:
                    return f"Weather query error: {str(e)}"

        class TextAnalysisWrapperPlugin:
            """Wrapper plugin for Text Analysis Agent."""
            
            @kernel_function(description="Analyze text for statistics, sentiment, and insights.")
            async def analyze_text(
                self, 
                text: Annotated[str, "The text to analyze"]
            ) -> Annotated[str, "Text analysis results and statistics"]:
                """Analyze text via the text analysis specialist."""
                try:
                    query = f"Please analyze this text: {text}"
                    response = await text_agent.get_response(messages=query)
                    return f"Text Analysis Specialist: {response.content}"
                except Exception as e:
                    return f"Text analysis error: {str(e)}"
        
        # Create plugin instances
        weather_plugin_wrapper = WeatherWrapperPlugin()
        text_plugin_wrapper = TextAnalysisWrapperPlugin()
        
        # Create your custom router
        custom_router = ChatCompletionAgent(
            id="CustomRouter",
            name="CustomRouter",
            instructions="""You are a custom router for weather and text analysis queries.

Available specialists via wrapper plugins:
• Weather Plugin: Provides weather information for any city or location
• Text Analysis Plugin: Analyzes text for statistics, sentiment, and insights

Route queries based on content:
- Weather queries: Use weather plugin for location-based weather requests
- Text analysis queries: Use text analysis plugin for text processing requests

Always indicate which specialist handled the query.""",
            plugins=[weather_plugin_wrapper, text_plugin_wrapper],
            service=AzureChatCompletion()
        )
        
        print("✅ Custom routing system created!")
        print("🔧 Using proper wrapper plugins for AzureAIAgent integration")
        
        # Test your system with different query types
        test_queries = [
            "What's the weather like in Seattle?",
            "Can you analyze this text: 'The quick brown fox jumps over the lazy dog. This is a test sentence for analysis.'"
        ]
        
        for i, test_query in enumerate(test_queries, 1):
            print(f"\n🧪 Test {i}: {test_query}")
            
            async for response in custom_router.invoke(messages=test_query):
                print(f"🤖 Custom Router: {response.content}")
                break
            
        # Cleanup custom agents
        await client.agents.delete_agent(weather_agent.id)
        await client.agents.delete_agent(text_agent.id)
        print("🧹 Custom agents cleaned up")
        
    except Exception as e:
        print(f"❌ Error in custom routing: {e}")

# Try the exercise
await create_custom_routing_system()

print("\n💡 Exercise Tips:")
print("1. Always create wrapper plugins with @kernel_function decorators")
print("2. AzureAIAgents cannot be used directly as plugins")
print("3. Use descriptive function names and descriptions")
print("4. Handle errors gracefully in wrapper functions")
print("5. Test with clear domain-specific queries")
print("6. Observe how routing decisions are made through wrapper plugins")

✅ All utility plugins defined!
🔧 Available plugins: Weather, Random, DateTime, TextAnalysis, RAG, NL2SQL, Cosmos
🎨 Creating Custom Routing System...
✅ Custom routing system created!
🔧 Using proper wrapper plugins for AzureAIAgent integration

🧪 Test 1: What's the weather like in Seattle?
🌤️ Weather check: Overcast and 65°F in Seattle
🤖 Custom Router: Weather Specialist: It’s overcast in Seattle right now, with the temperature around 65 °F.

🧪 Test 2: Can you analyze this text: 'The quick brown fox jumps over the lazy dog. This is a test sentence for analysis.'
📊 Text analysis complete: {'character_count': 82, 'word_count': 16, 'sentence_count': 2, 'average_word_length': 4.19}
🤖 Custom Router: Text Analysis Specialist handled your request.

Summary of findings:
• Characters (including spaces): 82  
• Words: 16  
• Sentences: 2  
• Average word length: 4.19 characters  

Let me know if you’d like deeper analysis such as readability scores, sentiment, or keyword extraction!
🧹 Custom agent

## 🧹 Step 5: Cleanup and Best Practices

**Important**: Always clean up your resources properly in agent routing systems!

**Cleanup Considerations:**
- **Multiple Agents**: More agents = more resources to clean up
- **Hierarchical Dependencies**: Clean up in proper order
- **Client Management**: Properly close connections

**Production Best Practices:**
- Monitor agent performance and costs
- Implement proper error handling for routing failures
- Use connection pooling for high-throughput scenarios
- Cache routing decisions for similar queries

In [9]:
# Comprehensive cleanup of all agents and resources

async def cleanup_routing_system():
    """Clean up all agents in the routing system."""
    
    print("🧹 Cleaning Up Agent Routing System")
    print("-" * 40)
    
    cleanup_count = 0
    
    if base_agents and base_agents['client']:
        client = base_agents['client']
        
        # Clean up base agents
        for agent_name, agent in base_agents.items():
            if agent_name != 'client' and hasattr(agent, 'id'):
                try:
                    await client.agents.delete_agent(agent.id)
                    print(f"✅ Deleted {agent_name} agent: {agent.name}")
                    cleanup_count += 1
                except Exception as e:
                    print(f"⚠️ Error deleting {agent_name} agent: {e}")
        
        # Close client connection
        try:
            await client.close()
            print("✅ Client connection closed")
        except Exception as e:
            print(f"ℹ️ Client cleanup: {e}")
    
    print(f"\n🎯 Cleanup Summary:")
    print(f"✅ {cleanup_count} base agents cleaned up")
    print(f"✅ Router agents (ChatCompletionAgent) automatically cleaned")
    print(f"✅ All connections properly closed")
    
    print("\n💡 Best Practices Demonstrated:")
    print("✅ Hierarchical cleanup (base agents first)")
    print("✅ Proper error handling during cleanup")
    print("✅ Resource management for complex systems")
    print("✅ Cost management through proper cleanup")

# Perform cleanup
await cleanup_routing_system()

🧹 Cleaning Up Agent Routing System
----------------------------------------
✅ Deleted insurance agent: insurance-rag-agent
✅ Deleted banking agent: banking-sql-agent
✅ Deleted sales agent: sales-analytics-agent
ℹ️ Client cleanup: 'AIProjectClient' object has no attribute 'aclose'

🎯 Cleanup Summary:
✅ 3 base agents cleaned up
✅ Router agents (ChatCompletionAgent) automatically cleaned
✅ All connections properly closed

💡 Best Practices Demonstrated:
✅ Hierarchical cleanup (base agents first)
✅ Proper error handling during cleanup
✅ Resource management for complex systems
✅ Cost management through proper cleanup


---

## 🎓 Congratulations! You're Now an Agent Routing Expert!

### What You've Mastered:

✅ **Hierarchical Agent Architecture:**
- AzureAIAgent for specialized, stateful base agents
- ChatCompletionAgent for lightweight routing and orchestration
- Multi-level agent coordination patterns

✅ **Intelligent Routing:**
- Domain-specific agent specialization
- Query analysis and intelligent dispatch
- Multi-agent coordination for complex queries

✅ **Production Patterns:**
- Proper resource management in complex systems
- Error handling across agent hierarchies
- Performance optimization for routing systems

### 🚀 Next Steps:

**Advanced Routing Patterns:**
1. **Conditional Routing**: Based on user context, preferences, or history
2. **Load Balancing**: Multiple instances of the same specialist type
3. **Fallback Chains**: Graceful degradation when specialists are unavailable
4. **Dynamic Agent Creation**: Creating specialists on-demand

**Enterprise Integration:**
- Authentication and authorization across agent hierarchies
- Monitoring and observability for routing decisions
- Cost optimization through intelligent agent utilization
- A/B testing different routing strategies

### 💡 Key Architectural Insights:

**When to Use AzureAIAgent vs ChatCompletionAgent:**
- **AzureAIAgent**: Complex workflows, state management, persistent conversations
- **ChatCompletionAgent**: Fast decisions, routing logic, stateless operations

**Scaling Patterns:**
- Start with simple 1-to-many routing
- Add coordination layers for complex workflows
- Implement caching and optimization as you scale

**Happy routing with Azure AI Agents!** 🎉

---

## 🔧 Quick Troubleshooting Guide

**Routing Issues:**
- Verify agent instructions are clear and domain-specific
- Check plugin integration for base agents
- Test individual agents before building hierarchies

**Performance Issues:**
- Monitor routing decision time vs. specialist execution time
- Consider caching for repeated routing patterns
- Use async operations throughout the hierarchy

**Cost Management:**
- Track token usage across multiple agents
- Implement query optimization to reduce unnecessary calls
- Clean up agents promptly to avoid ongoing charges